In [1]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [9]:
df = pd.read_csv("clean-data/Whiskey_data/Whiskey_Advocate_All_scraped_KHupdate-with-description.csv").drop(columns = 'row_other')

In [10]:
df.dropna(subset=['description', 'price'], inplace = True)

In [11]:
df.tail()

,row_caller,Maker,distilled,age in cast,ABV,Blended,Bourbon,Flavored,Other,Rye,Scotch,single blended grain,single blended malt,world,review score,price,style,country,description
2441,4642,The English Whisky Co.,NaN,NaN,53.40%,0,0,0,0,0,0,0,1,1,84,87,Single Malt English Whisky,UK,"Simple nose of corn, oak, cinnamon candy, and ..."
2442,4649,Teerenpeli,NaN,NaN,43.00%,0,0,0,0,0,0,0,1,1,84,87,Single Malt Finnish Whisky,FI,"Beautiful little bottle and label. 100% rye, a..."
2443,4655,Michel Couvreur,NaN,10 year old,45.00%,0,0,0,1,0,0,0,1,0,83,58,Unspecified Single Malt,UK,Grassy notes and damp hay define the aroma of ...
2444,4659,Woodford Reserve,NaN,NaN,45.20%,0,0,0,0,0,0,0,1,0,83,100,Single Malt American Whiskey,US,"As mentioned in my column this issue, Stillhou..."
2445,4744,Mackmyra,NaN,NaN,46.10%,0,0,0,0,0,0,0,1,1,83,64,Single Malt Swedish Whisky,SE,Showing its youth through an obvious green cha...


In [12]:
bins = [0, 75, 80, 85, 90, 95, 100]

In [92]:
df['binned scores'] = pd.qcut(df['price'], q = 5, labels = [0, 1, 2, 3, 4])

In [96]:
df['binned scores'].astype('int64')

0       3
1       4
2       4
3       4
4       3
5       4
6       4
7       4
8       4
9       4
10      4
11      4
12      2
13      2
14      4
15      1
16      4
17      4
18      2
19      4
20      4
21      3
22      4
23      4
24      4
25      3
26      4
27      4
28      4
29      4
       ..
2416    2
2417    2
2418    1
2419    3
2420    1
2421    3
2422    2
2423    2
2424    1
2425    0
2426    3
2427    1
2428    3
2429    2
2430    3
2431    3
2432    3
2433    2
2434    2
2435    2
2436    3
2437    2
2438    2
2439    2
2440    2
2441    2
2442    2
2443    0
2444    2
2445    1
Name: binned scores, Length: 2446, dtype: int64

In [67]:
dummies = pd.get_dummies(df['binned scores'])

In [97]:
X = df['description']
y = df['binned scores'].astype('int64')

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 43, test_size = .1)

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

In [100]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', MultinomialNB(
                    fit_prior=True, class_prior=None)),
            ])

In [76]:
bins = df['binned scores'].unique()

In [135]:
NB_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [136]:
prediction = NB_pipeline.predict(X_test)

In [137]:
NB_pipeline.score(X_train, y_train)

0.8441617446615175

In [138]:
NB_pipeline.score(X_test, y_test)

0.2897959183673469

In [132]:
len(X_train)

2201

In [133]:
len(X_test)

245